# **UAS**

Link Streamlit : https://appppwberita-bicgzybnrpji956hvld6vc.streamlit.app/

Nama : Naufal Abdullah R.Z

NIM  : 200411100096



1. Crawling Berita
2. LDA (Representasi Fitur)
3. Model Klasifikasi
4. Evaluasi
5. Implementasi
    CRAWLING BERITA 1000 BERITA 3 KATEGORI

**Crawling**

Crawling web adalah proses pengumpulan data dari berbagai halaman web menggunakan program komputer yang disebut "web crawler" atau "spider." Secara otomatis, web crawler menjelajahi internet, mengikuti tautan antar halaman, dan mengumpulkan informasi dengan tujuan untuk diindeks atau disimpan.

**Algoritma LDA**

* **Inisialisasi Parameter:**

  Tentukan jumlah topik, $K$. Tetapkan distribusi prior Dirichlet untuk distribusi topik dalam dokumen $(α)$ dan distribusi kata dalam topik $(β)$.

* **Inisialisasi Alokasi Topik:**

  Berikan topik awal secara acak untuk setiap kata dalam setiap dokumen.

* **Iterasi Gibbs Sampling (atau Variational Inference):**

  *Ulangi langkah-langkah berikut untuk sejumlah iterasi tertentu:*

  - **Perhitungan Distribusi Posterior:**

    Hitung distribusi probabilitas posterior untuk setiap kata dalam setiap dokumen berdasarkan alokasi topik saat ini.

  - **Resampling Topik:**

    Alokasikan ulang setiap kata secara acak ke topik baru berdasarkan distribusi posterior dan distribusi kata dalam topik (β).

  - **Perhitungan Distribusi Posterior:**

    Hitung kembali distribusi probabilitas posterior setelah pengalokasian ulang topik.

* **Output Hasil:**

  Setelah sejumlah iterasi, hasil akhir adalah distribusi topik dalam setiap dokumen dan distribusi kata dalam setiap topik.

Meskipun langkah-langkahnya terlihat rumit, algoritma LDA umumnya menggunakan Gibbs Sampling atau Variational Inference untuk mencapai distribusi posterior yang sesuai. Dengan melakukan iterasi dan mengulangi langkah-langkah tersebut, model LDA berusaha untuk menemukan distribusi topik yang paling mungkin dalam korpus dokumen.

## Implementas
Saya Crawling data judul berita dan isi berita dari 2 website dikarenakan website pertama datanya terlalu sedikit jadi saya menggabungkan website www.detikinews.com dan www.beritaSatu.com

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Crawling dari detikinews.com

In [ ]:

# Daftar kategori yang dapat diisi custom sesuai kebutuhan
kategori = ['Politik','Sosial','Nasional']

# Jumlah data yang diambil per kategori
jumlah_data_per_kategori = 500

# List untuk menyimpan data
data_list = []

# Looping untuk setiap kategori
for kategori_terpilih in kategori:
    katakunci = kategori_terpilih

    # Looping untuk setiap halaman
    for halaman in range(1, jumlah_data_per_kategori // 10 + 1):  # 10 berita per halaman
        url = "https://www.detikinews.com/page/{}/?s={}".format(halaman, katakunci)
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10240'
        }
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        items = soup.findAll('div', 'listing listing-blog listing-blog-5 clearfix')

        # Looping untuk setiap berita dalam halaman tertentu
        for it in items:
            judul = it.find('a', 'post-title post-url').text.strip()

            tanggal_tag = it.find('span', 'time')
            tanggal = ''
            if tanggal_tag:
                tanggal = ' '.join(tanggal_tag.text.strip().split('\n'))

            link = it.find('a', 'post-title post-url')
            if link:
                href = link.get('href')
                response = requests.get(href, headers=headers)
                berita_soup = BeautifulSoup(response.text, 'html.parser')

                konten_tag = berita_soup.find('div', 'entry-content clearfix single-post-content')
                konten = ''
                if konten_tag:
                    konten = konten_tag.text.strip()

                # Menambahkan data ke dalam list
                data_list.append({'Kategori': katakunci, 'Judul': judul, 'Tanggal': tanggal, 'Link': href, 'Konten': konten})

# Membuat dataframe dari list data
df = pd.DataFrame(data_list)

# Menyimpan dataframe ke dalam file CSV
file_path = '/content/drive/MyDrive/penambangan_web/hasil/databerita.csv'
df.to_csv(file_path, index=False)

print("Data berhasil diambil.")

Data berhasil diambil.


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/penambangan_web/hasil/databerita.csv")
df1

,Kategori,Judul,Tanggal,Link,Konten
0,Politik,"Buka Pembekalan Adat, Bupati Rohil Ajak Calon ...","Jun 12, 2023",https://www.detikinews.com/buka-pembekalan-ada...,"Rokan Hilir, detikinews.com — Bupati Rokan Hil..."
1,Politik,DPRD Kepulauan Meranti Gelar Rapat Paripurna P...,"Nov 18, 2022",https://www.detikinews.com/dprd-kepulauan-mera...,"SELATPANJANG, detikinews.com – DPRD Kepulauan ..."
2,Politik,Bupati Siak bersama Forkopimda Ikuti Upacara H...,"Jun 1, 2022",https://www.detikinews.com/bupati-siak-bersama...,"SIAK, detikinews.com — Bupati Siak Alfedri, di..."
3,Politik,Bupati H.M. Adil Akan Salurkan Dana BLT Rp200r...,"Sep 24, 2021",https://www.detikinews.com/bupati-h-m-adil-aka...,"Meranti, detikinews.com — Sesuai Program dan J..."
4,Politik,Wakil Bupati Bagus Santoso Sambut Kunjungan Si...,"Mei 5, 2021",https://www.detikinews.com/wakil-bupati-bagus-...,"Bengkalis, Detikinews.com — Pengurus Dewan Pim..."
...,...,...,...,...,...
76,Nasional,Melalui Vidcom Bupati Kampar Ikuti Musrenbangn...,"Apr 30, 2020",https://www.detikinews.com/melalui-vidcom-bupa...,"BANGKINANG, Detikinews.com – Bupati Kampar H...."
77,Nasional,PLKB dan Kader Berperan Dalam Pencegahan Covid-19,"Apr 3, 2020",https://www.detikinews.com/plkb-dan-kader-berp...,"BANGKINANG, Detikunews.com – Petugas Lapangan..."
78,Nasional,Dashboard Lancang Kuning Akan Resmi Menjadi Ap...,"Mar 9, 2020",https://www.detikinews.com/dashboard-lancang-k...,"Pekanbaru, Detikinews.com — Panglima TNI Marse..."
79,Nasional,Polda Riau Amankan 3 Pelaku Perdagangan Kulit ...,"Feb 16, 2020",https://www.detikinews.com/polda-riau-amankan-...,"PEKANBARU, Detikinews.com – Polda Riau kembali..."


## Crawling dari www.beritaSatu.com

In [ ]:
# Daftar kategori yang dapat diisi custom sesuai kebutuhan
kategori = ['Pemilu', 'Sport', 'Techno']

# Jumlah data yang diambil per kategori
jumlah_data_per_kategori = 500

# List untuk menyimpan data
data_list = []

# Looping untuk setiap kategori
for kategori_terpilih in kategori:
    katakunci = kategori_terpilih

    # Looping untuk setiap halaman
    for halaman in range(1, jumlah_data_per_kategori // 20 + 1):  # 20 berita per halaman
        url = "https://www.beritasatu.com/search/{}/{}".format(katakunci, halaman)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0'
        }
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        items = soup.findAll('div', 'row mt-4 position-relative')

        # Looping untuk setiap berita dalam halaman tertentu
        for it in items:
            # Looping judul
            judul = it.find('h2', 'h5 fw-bold').text.strip()
            # Looping tanggal berita
            tanggal_tag = it.find('span', 'b1-date text-muted')
            tanggal = ''
            if tanggal_tag:
                tanggal = ' '.join(tanggal_tag.text.strip().split('\n'))
            # Looping link berita untuk diambil data kontennya
            link = it.find('a', 'stretched-link')
            if link:
                href = link.get('href')
                response = requests.get(href, headers=headers)
                berita_soup = BeautifulSoup(response.text, 'html.parser')
                #Looping untuk konten berita jika link bisa diakses
                konten_tag = berita_soup.find('div', 'col b1-article body-content')
                konten = ''
                if konten_tag:
                    konten = konten_tag.text.strip()

                # Menambahkan data ke dalam list
                data_list.append({'Kategori': katakunci, 'Judul': judul, 'Tanggal': tanggal, 'Link': href, 'Konten': konten})

# Membuat dataframe dari list data
df = pd.DataFrame(data_list)

# Menyimpan dataframe ke dalam file CSV
file_path = '/content/drive/MyDrive/penambangan_web/hasil/databerita2.csv'
df.to_csv(file_path, index=False)

print("Pengambilan data selesai.")


Pengambilan data selesai.


In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/penambangan_web/hasil/databerita2.csv")
df2

,Kategori,Judul,Tanggal,Link,Konten
0,Pemilu,"Prabowo Tampil Santai Saat Debat, Ridwan Kamil...",1 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com - Ketua Tim Kampanye D..."
1,Pemilu,BNPT: Harus Ada Tindakan Preventif Mengantisip...,1 jam yang lalu,https://www.beritasatu.com/nasional/2786748/bn...,"Jakarta, Beritasatu.com - Direktur Pencegahan ..."
2,Pemilu,"Hadapi Kampanye, Artis PAN Gelorakan Peningkat...",1 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com - Sejumlah artis Parta..."
3,Pemilu,"Debat Capres, TKN Puas Prabowo Tegas Ingatkan ...",1 jam yang lalu,https://www.beritasatu.com/bersatu-kawal-pemil...,"Jakarta, Beritasatu.com - Tim Kampanye Nasiona..."
4,Pemilu,"Pemilu 2024 Diperkirakan Kontribusikan Rp 294,...",8 jam yang lalu,https://www.beritasatu.com/ekonomi/2786675/pem...,"Jakarta, Beritasatu.com - Center of Reform on ..."
...,...,...,...,...,...
1232,Techno,Inilah Daftar Lengkap Pemenang AMI Awards 2012,5 Jul 2012 | 09:47 WIB,https://www.beritasatu.com/news/58200/inilah-d...,Agnes dan Afgan berbagi penghargaanrrAjang pen...
1233,Techno,Oka Sulaksana Tampil di ABG,7 Jun 2012 | 16:25 WIB,https://www.beritasatu.com/news/52678/oka-sula...,"Oka nantinya akan turun di kelas ""techno"" putr..."
1234,Techno,"Saat Pemakai Terangsang, Gaun ini Jadi Trasparan",5 Apr 2012 | 15:42 WIB,https://www.beritasatu.com/news/41000/saat-pem...,"Gaun ini segera dipasarkan.rrIntimacy 2.0, ada..."
1235,Techno,Situs Mobil Esemka Palsu Beredar,24 Feb 2012 | 13:43 WIB,https://www.beritasatu.com/ekonomi/33442/situs...,Pelaku membuat website palsu http://www.mobile...


##Gabungkan Data

Menggabungkan data dari kedua website yang sudah di crawling

In [ ]:
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df

,Kategori,Judul,Tanggal,Link,Konten
0,Politik,"Buka Pembekalan Adat, Bupati Rohil Ajak Calon ...","Jun 12, 2023",https://www.detikinews.com/buka-pembekalan-ada...,"Rokan Hilir, detikinews.com — Bupati Rokan Hil..."
1,Politik,DPRD Kepulauan Meranti Gelar Rapat Paripurna P...,"Nov 18, 2022",https://www.detikinews.com/dprd-kepulauan-mera...,"SELATPANJANG, detikinews.com – DPRD Kepulauan ..."
2,Politik,Bupati Siak bersama Forkopimda Ikuti Upacara H...,"Jun 1, 2022",https://www.detikinews.com/bupati-siak-bersama...,"SIAK, detikinews.com — Bupati Siak Alfedri, di..."
3,Politik,Bupati H.M. Adil Akan Salurkan Dana BLT Rp200r...,"Sep 24, 2021",https://www.detikinews.com/bupati-h-m-adil-aka...,"Meranti, detikinews.com — Sesuai Program dan J..."
4,Politik,Wakil Bupati Bagus Santoso Sambut Kunjungan Si...,"Mei 5, 2021",https://www.detikinews.com/wakil-bupati-bagus-...,"Bengkalis, Detikinews.com — Pengurus Dewan Pim..."
...,...,...,...,...,...
1313,Techno,Inilah Daftar Lengkap Pemenang AMI Awards 2012,5 Jul 2012 | 09:47 WIB,https://www.beritasatu.com/news/58200/inilah-d...,Agnes dan Afgan berbagi penghargaanrrAjang pen...
1314,Techno,Oka Sulaksana Tampil di ABG,7 Jun 2012 | 16:25 WIB,https://www.beritasatu.com/news/52678/oka-sula...,"Oka nantinya akan turun di kelas ""techno"" putr..."
1315,Techno,"Saat Pemakai Terangsang, Gaun ini Jadi Trasparan",5 Apr 2012 | 15:42 WIB,https://www.beritasatu.com/news/41000/saat-pem...,"Gaun ini segera dipasarkan.rrIntimacy 2.0, ada..."
1316,Techno,Situs Mobil Esemka Palsu Beredar,24 Feb 2012 | 13:43 WIB,https://www.beritasatu.com/ekonomi/33442/situs...,Pelaku membuat website palsu http://www.mobile...


In [ ]:
combined_df['Kategori'].value_counts()

Sport       500
Pemilu      498
Techno      239
Sosial       42
Nasional     32
Politik       7
Name: Kategori, dtype: int64

In [ ]:
# Save the DataFrame to a CSV file
combined_df.to_csv('/content/drive/MyDrive/penambangan_web/hasil/berita_gabungan.csv', index=False)

In [ ]:
combined_df

,Kategori,Judul,Tanggal,Link,Konten
0,Politik,"Buka Pembekalan Adat, Bupati Rohil Ajak Calon ...","Jun 12, 2023",https://www.detikinews.com/buka-pembekalan-ada...,"Rokan Hilir, detikinews.com — Bupati Rokan Hil..."
1,Politik,DPRD Kepulauan Meranti Gelar Rapat Paripurna P...,"Nov 18, 2022",https://www.detikinews.com/dprd-kepulauan-mera...,"SELATPANJANG, detikinews.com – DPRD Kepulauan ..."
2,Politik,Bupati Siak bersama Forkopimda Ikuti Upacara H...,"Jun 1, 2022",https://www.detikinews.com/bupati-siak-bersama...,"SIAK, detikinews.com — Bupati Siak Alfedri, di..."
3,Politik,Bupati H.M. Adil Akan Salurkan Dana BLT Rp200r...,"Sep 24, 2021",https://www.detikinews.com/bupati-h-m-adil-aka...,"Meranti, detikinews.com — Sesuai Program dan J..."
4,Politik,Wakil Bupati Bagus Santoso Sambut Kunjungan Si...,"Mei 5, 2021",https://www.detikinews.com/wakil-bupati-bagus-...,"Bengkalis, Detikinews.com — Pengurus Dewan Pim..."
...,...,...,...,...,...
1313,Techno,Inilah Daftar Lengkap Pemenang AMI Awards 2012,5 Jul 2012 | 09:47 WIB,https://www.beritasatu.com/news/58200/inilah-d...,Agnes dan Afgan berbagi penghargaanrrAjang pen...
1314,Techno,Oka Sulaksana Tampil di ABG,7 Jun 2012 | 16:25 WIB,https://www.beritasatu.com/news/52678/oka-sula...,"Oka nantinya akan turun di kelas ""techno"" putr..."
1315,Techno,"Saat Pemakai Terangsang, Gaun ini Jadi Trasparan",5 Apr 2012 | 15:42 WIB,https://www.beritasatu.com/news/41000/saat-pem...,"Gaun ini segera dipasarkan.rrIntimacy 2.0, ada..."
1316,Techno,Situs Mobil Esemka Palsu Beredar,24 Feb 2012 | 13:43 WIB,https://www.beritasatu.com/ekonomi/33442/situs...,Pelaku membuat website palsu http://www.mobile...


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/penambangan_web/hasil/berita_gabungan.csv")
df

,Kategori,Judul,Tanggal,Link,Konten
0,Politik,"Buka Pembekalan Adat, Bupati Rohil Ajak Calon ...","Jun 12, 2023",https://www.detikinews.com/buka-pembekalan-ada...,"Rokan Hilir, detikinews.com — Bupati Rokan Hil..."
1,Politik,DPRD Kepulauan Meranti Gelar Rapat Paripurna P...,"Nov 18, 2022",https://www.detikinews.com/dprd-kepulauan-mera...,"SELATPANJANG, detikinews.com – DPRD Kepulauan ..."
2,Politik,Bupati Siak bersama Forkopimda Ikuti Upacara H...,"Jun 1, 2022",https://www.detikinews.com/bupati-siak-bersama...,"SIAK, detikinews.com — Bupati Siak Alfedri, di..."
3,Politik,Bupati H.M. Adil Akan Salurkan Dana BLT Rp200r...,"Sep 24, 2021",https://www.detikinews.com/bupati-h-m-adil-aka...,"Meranti, detikinews.com — Sesuai Program dan J..."
4,Politik,Wakil Bupati Bagus Santoso Sambut Kunjungan Si...,"Mei 5, 2021",https://www.detikinews.com/wakil-bupati-bagus-...,"Bengkalis, Detikinews.com — Pengurus Dewan Pim..."
...,...,...,...,...,...
1313,Techno,Inilah Daftar Lengkap Pemenang AMI Awards 2012,5 Jul 2012 | 09:47 WIB,https://www.beritasatu.com/news/58200/inilah-d...,Agnes dan Afgan berbagi penghargaanrrAjang pen...
1314,Techno,Oka Sulaksana Tampil di ABG,7 Jun 2012 | 16:25 WIB,https://www.beritasatu.com/news/52678/oka-sula...,"Oka nantinya akan turun di kelas ""techno"" putr..."
1315,Techno,"Saat Pemakai Terangsang, Gaun ini Jadi Trasparan",5 Apr 2012 | 15:42 WIB,https://www.beritasatu.com/news/41000/saat-pem...,"Gaun ini segera dipasarkan.rrIntimacy 2.0, ada..."
1316,Techno,Situs Mobil Esemka Palsu Beredar,24 Feb 2012 | 13:43 WIB,https://www.beritasatu.com/ekonomi/33442/situs...,Pelaku membuat website palsu http://www.mobile...


## Cek Data Null

In [ ]:
df.isnull().sum()

Kategori    0
Judul       0
Tanggal     0
Link        0
Konten      3
dtype: int64

## Hapus Data Null

In [ ]:
df = df.dropna()
df.isnull().sum()

Kategori    0
Judul       0
Tanggal     0
Link        0
Konten      0
dtype: int64

## Import library

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import warnings
import pandas as pd
import numpy as np
import nltk
import re
import csv

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Punctuation


Punctuation mengacu pada proses menghilangkan tanda baca dalam sebuah teks, seperti koma (,), titik (.), tanda seru (!), tanda tanya (?), dan karakter serupa.

In [ ]:
df['Punctuation'] = df['Konten'].str.replace(r'[^\w\s,.?!]', '', regex=True).str.replace('\d+', '', regex=True).str.lower()
df

,Kategori,Judul,Tanggal,Link,Konten,Punctuation
0,Politik,"Buka Pembekalan Adat, Bupati Rohil Ajak Calon ...","Jun 12, 2023",https://www.detikinews.com/buka-pembekalan-ada...,"Rokan Hilir, detikinews.com — Bupati Rokan Hil...","rokan hilir, detikinews.com bupati rokan hili..."
1,Politik,DPRD Kepulauan Meranti Gelar Rapat Paripurna P...,"Nov 18, 2022",https://www.detikinews.com/dprd-kepulauan-mera...,"SELATPANJANG, detikinews.com – DPRD Kepulauan ...","selatpanjang, detikinews.com dprd kepulauan m..."
2,Politik,Bupati Siak bersama Forkopimda Ikuti Upacara H...,"Jun 1, 2022",https://www.detikinews.com/bupati-siak-bersama...,"SIAK, detikinews.com — Bupati Siak Alfedri, di...","siak, detikinews.com bupati siak alfedri, did..."
3,Politik,Bupati H.M. Adil Akan Salurkan Dana BLT Rp200r...,"Sep 24, 2021",https://www.detikinews.com/bupati-h-m-adil-aka...,"Meranti, detikinews.com — Sesuai Program dan J...","meranti, detikinews.com sesuai program dan ja..."
4,Politik,Wakil Bupati Bagus Santoso Sambut Kunjungan Si...,"Mei 5, 2021",https://www.detikinews.com/wakil-bupati-bagus-...,"Bengkalis, Detikinews.com — Pengurus Dewan Pim...","bengkalis, detikinews.com pengurus dewan pimp..."
...,...,...,...,...,...,...
1313,Techno,Inilah Daftar Lengkap Pemenang AMI Awards 2012,5 Jul 2012 | 09:47 WIB,https://www.beritasatu.com/news/58200/inilah-d...,Agnes dan Afgan berbagi penghargaanrrAjang pen...,agnes dan afgan berbagi penghargaanrrajang pen...
1314,Techno,Oka Sulaksana Tampil di ABG,7 Jun 2012 | 16:25 WIB,https://www.beritasatu.com/news/52678/oka-sula...,"Oka nantinya akan turun di kelas ""techno"" putr...","oka nantinya akan turun di kelas techno putra,..."
1315,Techno,"Saat Pemakai Terangsang, Gaun ini Jadi Trasparan",5 Apr 2012 | 15:42 WIB,https://www.beritasatu.com/news/41000/saat-pem...,"Gaun ini segera dipasarkan.rrIntimacy 2.0, ada...","gaun ini segera dipasarkan.rrintimacy ., adala..."
1316,Techno,Situs Mobil Esemka Palsu Beredar,24 Feb 2012 | 13:43 WIB,https://www.beritasatu.com/ekonomi/33442/situs...,Pelaku membuat website palsu http://www.mobile...,pelaku membuat website palsu httpwww.mobilesem...


##TOKENISASI

In [ ]:
df['Tokenizing'] = df['Punctuation'].apply(lambda text: word_tokenize(text.lower()))
df['Tokenizing']

0       [rokan, hilir, ,, detikinews.com, bupati, roka...
1       [selatpanjang, ,, detikinews.com, dprd, kepula...
2       [siak, ,, detikinews.com, bupati, siak, alfedr...
3       [meranti, ,, detikinews.com, sesuai, program, ...
4       [bengkalis, ,, detikinews.com, pengurus, dewan...
                              ...                        
1313    [agnes, dan, afgan, berbagi, penghargaanrrajan...
1314    [oka, nantinya, akan, turun, di, kelas, techno...
1315    [gaun, ini, segera, dipasarkan.rrintimacy, ., ...
1316    [pelaku, membuat, website, palsu, httpwww.mobi...
1317    [mobil, esemka, ini, berangkat, dari, solo, me...
Name: Tokenizing, Length: 1315, dtype: object

##STOPWORDS REMOVAL

In [ ]:
# Daftar stopwords dalam bahasa indonesia dari NLTK
stop_words = set(stopwords.words('indonesian'))

# Stopwords
df['Stopword'] = df['Punctuation'].apply(lambda text: [word for word in word_tokenize(text.lower()) if word.isalnum() and word not in stop_words])


# Menggabungkan teks
df['JoinText'] = df['Stopword'].apply(lambda x: ' '.join(x))
df['JoinText']

0       rokan hilir bupati rokan hilir rohil afrizal s...
1       selatpanjang dprd kepulauan meranti melaksanak...
2       siak bupati siak alfedri didampingi wakil bupa...
3       meranti sesuai program janji bupati kepulauan ...
4       bengkalis pengurus dewan pimpinan cabang dpc p...
                              ...                        
1313    agnes afgan berbagi penghargaanrrajang penghar...
1314    oka turun kelas techno putra r ratlet selancar...
1315    gaun nama gaun terbuat kain cerdas transparan ...
1316    pelaku website palsu agen penjualan mobil kiat...
1317    mobil esemka berangkat solo melewati rute pant...
Name: JoinText, Length: 1315, dtype: object

##TERM FREQUENCY

Untuk melihat nilai seberapa penting sebuah kata dalam suatu dokumen didalam corpus

In [ ]:
# Inisialisasi TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transformasi TF-IDF di kolom JoinText
matrix_tfidf = tfidf_vectorizer.fit_transform(df['JoinText'])
term = tfidf_vectorizer.get_feature_names_out()

# Inisialisasi DF dari TF IDF
df_tfidf = pd.DataFrame(matrix_tfidf.toarray(), columns=term)
tfidf_data = pd.concat([df['JoinText'],df_tfidf,df['Kategori']],axis=1)

# Menghapus NaN jika ada
tfidf_data = tfidf_data.dropna(axis=0)

In [ ]:
tfidf_data

,JoinText,aa,aagn,aal,aam,aamanat,aan,aandal,aang,aarm,...,zulkieflimansyah,zulkifli,zulmi,zumba,zumi,zuriana,zvonimir,âœgongâ,âœhonda,Kategori
0,rokan hilir bupati rokan hilir rohil afrizal s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
1,selatpanjang dprd kepulauan meranti melaksanak...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
2,siak bupati siak alfedri didampingi wakil bupa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
3,meranti sesuai program janji bupati kepulauan ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
4,bengkalis pengurus dewan pimpinan cabang dpc p...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Politik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,pidato bernanke gairahkan perdagangan saham wa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Techno
1311,detil faktor berpengaruh penurunan harga komod...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Techno
1312,churchill menguasai persen proyek batu bara ku...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Techno
1313,agnes afgan berbagi penghargaanrrajang penghar...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Techno


## Save ke csv

In [ ]:
# Save the DataFrame to a CSV file
tfidf_data.to_csv("/content/drive/MyDrive/penambangan_web/hasil/tfidf.csv")

## Save Model

In [ ]:
import joblib

joblib.dump(tfidf_vectorizer,'/content/drive/MyDrive/penambangan_web/hasil/Model/tfidf_vectorizer.pkl')

['/content/drive/MyDrive/penambangan_web/hasil/Model/tfidf_vectorizer.pkl']

## Memisah X dan Y

Menentukan fitur dan target. Pada pembahasan kali ini `fitur = (X)` dan `target = (y)`.


> `X` didapatkan dari penghapusan kolom JoinText dan juga Kategori


> `y` didapatkan dari kolom Kategori pada dataframe sebagai target

In [ ]:
X = tfidf_data.drop(['JoinText','Kategori'], axis=1)
X

,aa,aagn,aal,aam,aamanat,aan,aandal,aang,aarm,aaron,...,zulkenedi,zulkieflimansyah,zulkifli,zulmi,zumba,zumi,zuriana,zvonimir,âœgongâ,âœhonda
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y = tfidf_data['Kategori']
y

0       Politik
1       Politik
2       Politik
3       Politik
4       Politik
         ...   
1310     Techno
1311     Techno
1312     Techno
1313     Techno
1314     Techno
Name: Kategori, Length: 1312, dtype: object

## Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelling

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import LatentDirichletAllocation

## Inisialisasi K, Alpha, Beta

In [ ]:
# Topik K yang akan diuji
k = range(3,20)
# Alpha yang akan diuji
alpha = [0.1]
# Beta yang akan diuji
beta = [0.2]

## Mencari hasil terbaik modelling LDA

In [ ]:
best_results = {'model_name': 'SVM', 'k': 0, 'alpha': 0, 'beta': 0, 'accuracy': 0, 'lda_train': None, 'lda_test': None}
best_lda_model = {'lda_model': None}
best_classifier_model = {'classifier_model': None}
all_results = []

for num_topics in k:
    for alpha_val in alpha:
        for beta_val in beta:
            # Inisialisasi model LDA dengan parameter tertentu
            lda = LatentDirichletAllocation(n_components=num_topics, doc_topic_prior=alpha_val, topic_word_prior=beta_val)

            # Latih model LDA pada training set yang telah divektorisasi
            lda_train_data = lda.fit_transform(X_train)
            lda_test_data = lda.transform(X_test)

            # Inisialisasi model klasifikasi SVM
            current_classifier_model = SVC()

            # Latih model klasifikasi pada data pelatihan
            current_classifier_model.fit(lda_train_data, y_train)

            # Lakukan prediksi pada data pengujian
            y_pred_test = current_classifier_model.predict(lda_test_data)

            # Hitung akurasi
            current_accuracy = accuracy_score(y_test, y_pred_test)

            print(f"Jumlah Topik: {num_topics}, Alpha: {alpha_val}, Beta: {beta_val}, Accuracy: {current_accuracy}")

            # Simpan hasil
            all_results.append({
                'Model': 'SVM',
                'Jumlah Topik': num_topics,
                'Alpha': alpha_val,
                'Beta': beta_val,
                'Accuracy': current_accuracy
            })

            # Cek akurasi tertinggi untuk best_results
            if current_accuracy > best_results['accuracy']:
                best_results['k'] = num_topics
                best_results['alpha'] = alpha_val
                best_results['beta'] = beta_val
                best_results['accuracy'] = current_accuracy
                best_results['lda_test'] = lda_test_data
                best_results['lda_train'] = lda_train_data
                best_lda_model['lda_model'] = lda
                best_classifier_model['classifier_model'] = current_classifier_model

# Tampilkan hasil terbaik
print("Best Results:")
print(best_results)


Jumlah Topik: 3, Alpha: 0.1, Beta: 0.2, Accuracy: 0.623574144486692
Jumlah Topik: 4, Alpha: 0.1, Beta: 0.2, Accuracy: 0.596958174904943
Jumlah Topik: 5, Alpha: 0.1, Beta: 0.2, Accuracy: 0.49049429657794674
Jumlah Topik: 6, Alpha: 0.1, Beta: 0.2, Accuracy: 0.5057034220532319
Jumlah Topik: 7, Alpha: 0.1, Beta: 0.2, Accuracy: 0.49429657794676807
Jumlah Topik: 8, Alpha: 0.1, Beta: 0.2, Accuracy: 0.5133079847908745
Jumlah Topik: 9, Alpha: 0.1, Beta: 0.2, Accuracy: 0.6349809885931559
Jumlah Topik: 10, Alpha: 0.1, Beta: 0.2, Accuracy: 0.42585551330798477
Jumlah Topik: 11, Alpha: 0.1, Beta: 0.2, Accuracy: 0.4752851711026616
Jumlah Topik: 12, Alpha: 0.1, Beta: 0.2, Accuracy: 0.4600760456273764
Jumlah Topik: 13, Alpha: 0.1, Beta: 0.2, Accuracy: 0.4448669201520912
Jumlah Topik: 14, Alpha: 0.1, Beta: 0.2, Accuracy: 0.3612167300380228
Jumlah Topik: 15, Alpha: 0.1, Beta: 0.2, Accuracy: 0.55893536121673
Jumlah Topik: 16, Alpha: 0.1, Beta: 0.2, Accuracy: 0.623574144486692
Jumlah Topik: 17, Alpha: 0.1,

In [ ]:
history = pd.DataFrame(all_results)
history

,Model,Jumlah Topik,Alpha,Beta,Accuracy
0,SVM,3,0.1,0.2,0.623574
1,SVM,4,0.1,0.2,0.596958
2,SVM,5,0.1,0.2,0.490494
3,SVM,6,0.1,0.2,0.505703
4,SVM,7,0.1,0.2,0.494297
5,SVM,8,0.1,0.2,0.513308
6,SVM,9,0.1,0.2,0.634981
7,SVM,10,0.1,0.2,0.425856
8,SVM,11,0.1,0.2,0.475285
9,SVM,12,0.1,0.2,0.460076


In [ ]:
history.to_csv('/content/drive/MyDrive/penambangan_web/hasil/historysvm.csv')

## Hasil terbaik disimpan

In [ ]:
df_bestParam = pd.DataFrame([best_results])
df_bestParam.to_csv("/content/drive/MyDrive/penambangan_web/hasil/parameterterbaik.csv",index=False)
df_bestParam

,model_name,k,alpha,beta,accuracy,lda_train,lda_test
0,SVM,9,0.1,0.2,0.634981,"[[0.010751263301723862, 0.010751067390055635, ...","[[0.013455439497835358, 0.013455439637785812, ..."


## Simpan Modelling

In [ ]:
lda = best_lda_model['lda_model']
joblib.dump(lda,"/content/drive/MyDrive/penambangan_web/hasil/Model/svm_lda.pkl")

model = best_classifier_model['classifier_model']
joblib.dump(model,"/content/drive/MyDrive/penambangan_web/hasil/Model/svm_model.pkl")

['/content/drive/MyDrive/penambangan_web/hasil/Model/svm_model.pkl']

## Load LDA

In [ ]:
from joblib import load
lda_mod = load("/content/drive/MyDrive/penambangan_web/hasil/Model/svm_lda.pkl")

In [ ]:
k = best_results['k']
xtrain = best_results['lda_train']
xtest = best_results['lda_test']

## Proporsi Topik

In [ ]:
kolom = [f"Topik {i}" for i in range(1, k+1)]
dokumen = df['Konten']
proporsi_topik_dokumen = pd.DataFrame(xtrain,columns=kolom)
proporsi_topik_dokumen.insert(0,'Konten',dokumen)
proporsi_topik_dokumen.insert(len(proporsi_topik_dokumen.columns),"Kategori",df['Kategori'])
proporsi_topik_dokumen


,Konten,Topik 1,Topik 2,Topik 3,Topik 4,Topik 5,Topik 6,Topik 7,Topik 8,Topik 9,Kategori
0,"Rokan Hilir, detikinews.com — Bupati Rokan Hil...",0.010751,0.010751,0.913983,0.010751,0.010751,0.010759,0.010751,0.010751,0.010751,Politik
1,"SELATPANJANG, detikinews.com – DPRD Kepulauan ...",0.011753,0.011753,0.011753,0.011753,0.011753,0.905978,0.011753,0.011753,0.011753,Politik
2,"SIAK, detikinews.com — Bupati Siak Alfedri, di...",0.012512,0.012512,0.012512,0.012512,0.012513,0.899901,0.012512,0.012512,0.012512,Politik
3,"Meranti, detikinews.com — Sesuai Program dan J...",0.015231,0.015231,0.015231,0.015231,0.015231,0.878150,0.015231,0.015231,0.015231,Politik
4,"Bengkalis, Detikinews.com — Pengurus Dewan Pim...",0.010666,0.010666,0.010667,0.010667,0.010666,0.914668,0.010666,0.010667,0.010666,Politik
...,...,...,...,...,...,...,...,...,...,...,...
1044,"Jakarta, Beritasatu.com - Ketua Komisi X DPR R...",0.009040,0.009040,0.009047,0.009040,0.009040,0.927673,0.009039,0.009042,0.009040,Sport
1045,"Palembang, Beritasatu.com - Pengurus tim bola ...",0.227354,0.011435,0.011433,0.011431,0.011431,0.692624,0.011430,0.011431,0.011432,Sport
1046,"Jakarta, Beritasatu.com - Penyelenggara SEA Ga...",0.735451,0.012448,0.012449,0.012448,0.012448,0.177412,0.012448,0.012448,0.012448,Sport
1047,"Bogor, Beritasatu.com - Aksi video viral sepas...",0.009519,0.009519,0.009519,0.009519,0.009519,0.923847,0.009519,0.009519,0.009519,Sport


In [ ]:
proporsi_topik_dokumen.to_csv("/content/drive/MyDrive/penambangan_web/hasil/proporsi_topik_dokumen.csv",index=False)

## Evaluasi

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
use_model_predict = load("/content/drive/MyDrive/penambangan_web/hasil/Model/svm_lda.pkl")

In [ ]:
# Melakukan prediksi pada data pengujian
y_pred = best_classifier_model['classifier_model'].predict(best_results['lda_test'])

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print(f"Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.6349809885931559
Laporan Klasifikasi:
              precision    recall  f1-score   support

    Nasional       0.00      0.00      0.00        10
      Pemilu       0.55      0.97      0.70       104
      Sosial       0.00      0.00      0.00         7
       Sport       0.86      0.68      0.76        95
      Techno       0.50      0.02      0.04        47

    accuracy                           0.63       263
   macro avg       0.38      0.34      0.30       263
weighted avg       0.61      0.63      0.56       263



## Implementasi

In [ ]:
inputnews = ['Aksi video viral sepasang pelajar SMPN 1 Ciawi berdansa cantik di depan ratusan teman-temannya menjadi viral belakangan ini.']
vectorizerInp = tfidf_vectorizer.transform(inputnews)

input_lda = lda_mod.transform(vectorizerInp)
best_classifier_model['classifier_model'].predict(input_lda)

array(['Sport'], dtype=object)